import pandas as pd 

In [9]:
import pandas as pd 

In [10]:
#load the data into separate dataframes
schools_df = pd.read_csv("Resources/schools_complete.csv")
students_df = pd.read_csv("Resources/students_complete.csv")

# Merge datasets on "school_name". 
school_data = pd.merge(students_df, schools_df, how="left", on=["school_name", "school_name"])


In [11]:
#district summary calculations
total_schools = schools_df["school_name"].nunique()
total_students = students_df["Student ID"].count()
total_budget = schools_df["budget"].sum()
avg_math_score = students_df["math_score"].mean()
avg_reading_score = students_df["reading_score"].mean()
perc_pass_math = (students_df[students_df["math_score"] >= 70]["Student ID"].count() / total_students) * 100
perc_pass_reading = (students_df[students_df["reading_score"] >= 70]["Student ID"].count() / total_students) * 100
overall_pass = students_df[(students_df["math_score"] >= 70) & (students_df["reading_score"] >= 70)]["Student ID"].count() / total_students * 100

#Combine into df
distrct_summary=pd.DataFrame({
    "Total Schools": [total_schools],
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Math Score": [avg_math_score],
    "Average Reading Score": [avg_reading_score],
    "% Passing Math": [perc_pass_math],
    "% Passing Reading": [perc_pass_reading],
    "% Overall Passing": [overall_pass]
})

In [12]:
#School Summary
# Group by school name
school_group = school_data.groupby("school_name")

# School type for each school
school_type = school_group["type"].first()

# Total students for each school
total_student_per_school = school_group.size()

# Total budget for each school
total_budget_per_school = school_group["budget"].first()

# Budget per student for each school
per_student_budget = total_budget_per_school / total_student_per_school

# Average math score for each school
avg_math_score_school = school_group["math_score"].mean()

# Average reading score for each school
avg_reading_score_school = school_group["reading_score"].mean()

# Percentage of students passing math in each school
pass_math_school = school_data[school_data["math_score"] >= 70].groupby("school_name").size()
perc_pass_math_school = (pass_math_school / total_student_per_school) * 100

# Percentage of students passing reading in each school
pass_reading_school = school_data[school_data["reading_score"] >= 70].groupby("school_name").size()
perc_pass_reading_school = (pass_reading_school / total_student_per_school) * 100

# Percentage of students passing both math and reading in each school
overall_pass_school = school_data[(school_data["math_score"] >= 70) & (school_data["reading_score"] >= 70)].groupby("school_name").size()
overall_pass_school_perc = (overall_pass_school / total_student_per_school) * 100

# Combining all the metrics into a new DataFrame
per_school_summary = pd.DataFrame({
    "School Type": school_type,
    "Total Students": total_student_per_school,
    "Total School Budget": total_budget_per_school,
    "Per Student Budget": per_student_budget,
    "Average Math Score": avg_math_score_school,
    "Average Reading Score": avg_reading_score_school,
    "% Passing Math": perc_pass_math_school,
    "% Passing Reading": perc_pass_reading_school,
    "% Overall Passing": overall_pass_school_perc
})

In [13]:
#Highest Performing Schools
# Sort the schools by % Overall Passing in descending order 
top_schools = per_school_summary.sort_values("% Overall Passing", ascending=False).head(5)

# Display the top 5 schools
print(top_schools)


                    School Type  Total Students  Total School Budget  \
school_name                                                            
Cabrera High School     Charter            1858              1081356   
Thomas High School      Charter            1635              1043130   
Griffin High School     Charter            1468               917500   
Wilson High School      Charter            2283              1319574   
Pena High School        Charter             962               585858   

                     Per Student Budget  Average Math Score  \
school_name                                                   
Cabrera High School               582.0           83.061895   
Thomas High School                638.0           83.418349   
Griffin High School               625.0           83.351499   
Wilson High School                578.0           83.274201   
Pena High School                  609.0           83.839917   

                     Average Reading Score  % Passing

In [14]:
#Lowest performing schools
# Sort the schools by % Overall Passing in ascending order
bottom_schools = per_school_summary.sort_values("% Overall Passing").head(5)

# Display the bottom 5 schools
print(bottom_schools)


                      School Type  Total Students  Total School Budget  \
school_name                                                              
Rodriguez High School    District            3999              2547363   
Figueroa High School     District            2949              1884411   
Huang High School        District            2917              1910635   
Hernandez High School    District            4635              3022020   
Johnson High School      District            4761              3094650   

                       Per Student Budget  Average Math Score  \
school_name                                                     
Rodriguez High School               637.0           76.842711   
Figueroa High School                639.0           76.711767   
Huang High School                   655.0           76.629414   
Hernandez High School               652.0           77.289752   
Johnson High School                 650.0           77.072464   

                       Av

In [15]:
#Scores by School Spending 
# Define the spending bins and group names
spending_bins = [0, 585, 630, 645, 680]
group_names = ["<$585", "$585-630", "$630-645", "$645-680"]

# Categorize spending based on the bins
per_school_summary["Spending Ranges (Per Student)"] = pd.cut(per_school_summary["Per Student Budget"], spending_bins, labels=group_names)

# Calculate averages for desired columns
spending_math_scores = per_school_summary.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = per_school_summary.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = per_school_summary.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = per_school_summary.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = per_school_summary.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

# Assemble into DataFrame
spending_summary = pd.DataFrame({
    "Average Math Score": spending_math_scores,
    "Average Reading Score": spending_reading_scores,
    "% Passing Math": spending_passing_math,
    "% Passing Reading": spending_passing_reading,
    "% Overall Passing": overall_passing_spending
})


In [16]:
#Math Scores by Grade
# Filter by grade and group by school name
ninth_grade = school_data[school_data["grade"] == "9th"]
ms_ninth=ninth_grade.groupby("school_name")["math_score"].mean()

tenth_grade = school_data[school_data["grade"] == "10th"]
ms_tenth=tenth_grade.groupby("school_name")["math_score"].mean()

eleventh_grade = school_data[school_data["grade"] == "11th"]
ms_eleventh=eleventh_grade.groupby("school_name")["math_score"].mean()

twelfth_grade = school_data[school_data["grade"] == "12th"]
ms_twelfth=twelfth_grade.groupby("school_name")["math_score"].mean()


#Math scores by grade
math_scores_by_grade = pd.DataFrame({
    "9th": ms_ninth,
    "10th": ms_tenth,
    "11th": ms_eleventh,
    "12th": ms_twelfth
})

In [17]:
# Reading scores by grade
ninth_grade = school_data[school_data["grade"] == "9th"]
rs_ninth=ninth_grade.groupby("school_name")["reading_score"].mean()

tenth_grade = school_data[school_data["grade"] == "10th"]
rs_tenth=tenth_grade.groupby("school_name")["reading_score"].mean()

eleventh_grade = school_data[school_data["grade"] == "11th"]
rs_eleventh=eleventh_grade.groupby("school_name")["reading_score"].mean()

twelfth_grade = school_data[school_data["grade"] == "12th"]
rs_twelfth=twelfth_grade.groupby("school_name")["reading_score"].mean()

reading_scores_by_grade = pd.DataFrame({
    "9th": rs_ninth,
    "10th": rs_tenth,
    "11th":rs_eleventh,
    "12th": rs_twelfth
})

In [18]:
#Schools by School Size 
# Define the size bins and group names
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]


In [19]:
# Categorize sizes based on the bins
per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"], size_bins, labels=group_names)


In [20]:
per_school_summary.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15 entries, Bailey High School to Wright High School
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   School Type                    15 non-null     object  
 1   Total Students                 15 non-null     int64   
 2   Total School Budget            15 non-null     int64   
 3   Per Student Budget             15 non-null     float64 
 4   Average Math Score             15 non-null     float64 
 5   Average Reading Score          15 non-null     float64 
 6   % Passing Math                 15 non-null     float64 
 7   % Passing Reading              15 non-null     float64 
 8   % Overall Passing              15 non-null     float64 
 9   Spending Ranges (Per Student)  15 non-null     category
 10  School Size                    15 non-null     category
dtypes: category(2), float64(6), int64(2), object(1)
memory usage: 1.5+ KB


In [21]:
# Calculate averages for the desired columns
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
overall_passing_size = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()

In [22]:
# Assemble into DataFrame
size_summary = pd.DataFrame({
    "Average Math Score": size_math_scores,
    "Average Reading Score": size_reading_scores,
    "% Passing Math": size_passing_math,
    "% Passing Reading": size_passing_reading,
    "% Overall Passing": overall_passing_size
})


In [23]:
#Score by School Type 
# Group by type of school
type_math_scores = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
type_reading_scores = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
type_passing_math = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
type_passing_reading = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
overall_passing_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

# Assemble into DataFrame
type_summary = pd.DataFrame({
    "Average Math Score": type_math_scores,
    "Average Reading Score": type_reading_scores,
    "% Passing Math": type_passing_math,
    "% Passing Reading": type_passing_reading,
    "% Overall Passing": overall_passing_type
})



In [24]:
type_summary


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
